In [1]:
import os
import os.path
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from matplotlib import cm, colors
import pytorch_lightning as pl

from simclr_finetuner import SSLFineTuner
from simclr_module import SimCLR
from pointnet_module import SupervisedPointNet
from datasets.shapenet_parts.shapenet_parts import ShapeNetParts
from datasets.data_modules import PartSegmentationDataModule

from transforms import FineTuningTestDataTransform
from augmentations.augmentations import Rotation, Flip, Rescale

/home/shepherd/anaconda3/lib/python3.7/site-packages/pl_bolts/utils/warnings.py:32: UserWarning: You want to use `wandb` which is not installed yet, install it with `pip install wandb`.
  f' install it with `pip install {pypi_name}`.' + extra_text
/home/shepherd/anaconda3/lib/python3.7/site-packages/pl_bolts/utils/warnings.py:32: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
  f' install it with `pip install {pypi_name}`.' + extra_text


ImportError: /usr/lib/x86_64-linux-gnu/libavutil.so.55: version `LIBAVUTIL_55' not found (required by /home/shepherd/anaconda3/lib/python3.7/site-packages/cv2/cv2.cpython-37m-x86_64-linux-gnu.so)

In [ ]:
batch_size = 16
#transforms = [Rotation(),Flip(),Rescale()]
transforms = [Rotation()]

backbone_ckpt = 'checkpoints/simclr_best_21071416584428.ckpt'
ssl_finetuner_ckpt = 'checkpoints/fine_tuning_best_21071514122946.ckpt'

for transform in transforms:
    current_transform = FineTuningTestDataTransform([transform])
    dm = PartSegmentationDataModule(batch_size,test_transforms)
    seg_class_map = dm.seg_class_map
    backbone = SimCLR.load_from_checkpoint(backbone_ckpt)
    fine_tuner = SSLFineTuner.load_from_checkpoint(ssl_finetuner_ckpt, backbone=backbone, 
                                   seg_class_map=seg_class_map)
    test_dataloader = dm.test_dataloader()
    trainer = pl.Trainer(
        gpus=1,
        num_nodes=1,
        precision=16,
        max_epochs=1,
        distributed_backend=None,
        sync_batchnorm=False,
    )
    results = trainer.test(model = fine_tuner, test_dataloaders = test_dataloader)
    print('Results for transform: {} \n{}'.format(str(transform),results))